<a href="https://colab.research.google.com/github/OscarBedford/MLCourse_Weekly_Exercises/blob/main/Exercise3_10_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

10. analogous to task 5, but using the 43 different Craddock region atlas resolutions (.scorr_mean) instead of the 10 region numbers from Dataset A.

Use sklearn.linear_model.LogisticRegression(hyperparameter set to
1.0) to fit separate logistic predictive models based on 100, 200, 300, …, 900, 1000
brain regions of interest (see ‘n_rois’) based on the first 80% of the 100 structural
brain scans to predict sex differences. Use the probabilistic prediction of LogReg
(predict_proba function) from the training data from the 10 already obtained
LogReg models to fit a LogReg stacking model to predict sex. Next, evaluate the
final LogReg stacking model on the unseen final 20% of the 100 structural brain
scans and print classification accuracy in XX.YY%.

In [1]:
!pip install nilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.6 MB 5.9 MB/s 


In [2]:
import numpy as np 
from nilearn import datasets 
from nilearn.maskers import NiftiLabelsMasker 
from nilearn.image import index_img 
import nibabel as nib

In [3]:
brain_data = datasets.fetch_oasis_vbm(n_subjects=100)
crad = datasets.fetch_atlas_craddock_2012()
atlas_nii = index_img(crad['scorr_mean'], 1) 
# This needs to be modified for some tasks
masker = NiftiLabelsMasker(labels_img= atlas_nii, standardize=True)
input_variables = masker.fit_transform(brain_data.gray_matter_maps)
output_variable = np.array(brain_data.ext_vars.mf == 'F', dtype=np.int) #gives 1 for females and 0 for males


Dataset created in /root/nilearn_data/oasis1



Downloaded 885399552 of 905208634 bytes (97.8%,    0.7s remaining) ...done. (32 seconds, 0 min)
Extracting data from /root/nilearn_data/oasis1/adcb228c7b26ed4f4fba93bd32bf8d97/archive_dartel.tgz..... done.
/usr/local/lib/python3.7/dist-packages/nilearn/datasets/struct.py:774: UserWarning: `legacy_format` will default to `False` in release 0.11. Dataset fetchers will then return pandas dataframes by default instead of recarrays.
  warnings.warn(_LEGACY_FORMAT_MSG)



Dataset created in /root/nilearn_data/craddock_2012



 ...done. (1 seconds, 0 min)
Extracting data from /root/nilearn_data/craddock_2012/64ec9dc239add5cb58c7248a4136b852/craddock_2011_parcellations.tar.gz..... done.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys


In [4]:
import matplotlib.pyplot as plt
import pandas as pd
from numpy import mean
from numpy import std
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.ensemble import StackingClassifier

In [5]:
versions = list(range(0,43))
training_preds = [] # Here we will store the probability classes of the 10 training sets
test_preds = [] # Here we will store the probability classes of the 10 test sets

In [6]:
for i in versions:
    brain_data = datasets.fetch_oasis_vbm(n_subjects=100)
    crad = datasets.fetch_atlas_craddock_2012()
    atlas_nii = index_img(crad['scorr_mean'], (i)) # We will iterate through the 43 atlas versions
    masker = NiftiLabelsMasker(labels_img= atlas_nii, standardize=True)
    input_variables = masker.fit_transform(brain_data.gray_matter_maps)
    output_variable = np.array(brain_data.ext_vars.mf == 'F', dtype=np.int) #gives 1 for females and 0 for males

    X, y = input_variables, output_variable
    X_scaled = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)

    print('Atlas version number: ',i)

    LR_proba = LogisticRegression(max_iter=1e4).fit(X_train,y_train) # We fit the LR learning model
    preds = LR_proba.predict_proba(X_train) # We define the training model and store its probability classifications as 'preds'
    training_preds.append(preds) # This is where we will iteratively store the scores for the training set probability class predictions
    score = LR_proba.score(X_train, y_train) # We will also export the score (...)
    score = np.multiply(score, 100) 
    print('Training accuracy: %.2f' % ((score))) # (...) and we will print it

    yhat = LR_proba.predict_proba(X_test) # We define the prediction model and store its prediction as "yhat"
    test_preds.append(yhat) # This is where we will store the test set class predictions
    scores = LR_proba.score(X_test, y_test) # We store the scores
    scores = np.multiply(scores, 100) 
    print('Test accuracy: %.2f' % ((scores))) # We print the scores

/usr/local/lib/python3.7/dist-packages/nilearn/datasets/struct.py:774: UserWarning: `legacy_format` will default to `False` in release 0.11. Dataset fetchers will then return pandas dataframes by default instead of recarrays.
  warnings.warn(_LEGACY_FORMAT_MSG)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys
/usr/local/lib/python3.7/dist-packages/nilearn/datasets/struct.py:774: UserWarning: `legacy_format` will default to `False` in release 0.11. Dataset fetchers will then r

Atlas version number:  0
Training accuracy: 77.50
Test accuracy: 70.00


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys
/usr/local/lib/python3.7/dist-packages/nilearn/datasets/struct.py:774: UserWarning: `legacy_format` will default to `False` in release 0.11. Dataset fetchers will then return pandas dataframes by default instead of recarrays.
  warnings.warn(_LEGACY_FORMAT_MSG)


Atlas version number:  1
Training accuracy: 85.00
Test accuracy: 60.00


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys
/usr/local/lib/python3.7/dist-packages/nilearn/datasets/struct.py:774: UserWarning: `legacy_format` will default to `False` in release 0.11. Dataset fetchers will then return pandas dataframes by default instead of recarrays.
  warnings.warn(_LEGACY_FORMAT_MSG)


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys
/usr/local/lib/python3.7/dist-packages/nilearn/datasets/struct.py:774: UserWarning: `legacy_format` will default to `False` in release 0.11. Dataset fetchers will then return pandas dataframes by default instead of recarrays.
  warnings.warn(_LEGACY_FORMAT_MSG)


Atlas version number:  40
Training accuracy: 100.00
Test accuracy: 70.00


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys
/usr/local/lib/python3.7/dist-packages/nilearn/datasets/struct.py:774: UserWarning: `legacy_format` will default to `False` in release 0.11. Dataset fetchers will then return pandas dataframes by default instead of recarrays.
  warnings.warn(_LEGACY_FORMAT_MSG)


Atlas version number:  41
Training accuracy: 100.00
Test accuracy: 65.00
Atlas version number:  42
Training accuracy: 100.00
Test accuracy: 65.00


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys


In [7]:
# Before we can compute the stacking model, we need to reshape the lists we got
training_preds = np.array(training_preds) # Convert output to numpy array for ease of manipulation
test_preds = np.array(test_preds) # ditto
training_preds = np.delete(training_preds, 1, 2) # We delete column 2 (index 1) from the third axis (index 2) because it's redundant
test_preds =  np.delete(test_preds, 1, 2) #ditto

training_preds = np.squeeze(training_preds) # We eliminate the extraneous axis of size 1
test_preds = np.squeeze(test_preds) #ditto
training_preds = np.transpose(training_preds) # We transpose the array so that rows (observations) are first and columns (features) are second
test_preds = np.transpose(test_preds) #ditto

In [8]:
 # We're ready to compute the stacking (or meta-) model:   
LR_stack = LogisticRegression(max_iter=1e4).fit(training_preds, y_train) # We fit the LR learning model
score = LR_stack.score(training_preds, y_train) # We will also export the score (...)
score = np.multiply(score, 100) 
print('Meta-model training accuracy: %.2f' % ((score))) # (...) and we will print it
   
yhat_stack = LR_stack.predict_proba(test_preds) # We define the prediction model and store its prediction as "yhat"
scores = LR_stack.score(test_preds, y_test) # We store the test set scores (...)
scores = np.multiply(scores, 100) 
print('Meta-model test accuracy: %.2f' % ((scores))) # (...) and we print the them

Meta-model training accuracy: 100.00
Meta-model test accuracy: 65.00


Voilà! We obtain a test set classification accuracy of 65% for the stacking (or meta-) model.